<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipdb
!pip install -U spacy
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 62.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninst

In [2]:
import spacy
import torch
import matplotlib.pyplot as plt
import ipdb
import numpy as np
import random
import pandas as pd
# %pdb on

In [3]:
# Helper function to plot the training metrics

def plot_training_metrics(train_acc, val_acc, train_loss, title, save_path):
    # Ensure that all input lists have the same length
    assert len(train_acc) == len(val_acc) == len(train_loss), "All input histories must have the same length."

    epochs = range(1, len(train_acc) + 1)

    # Create the metrics DataFrame
    df_metrics = pd.DataFrame({
        'Epoch': epochs,
        'Training Accuracy (%)': train_acc,
        'Validation Accuracy (%)': val_acc,
        'Training Loss': train_loss
    })

    # Initialize the plot
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot Training and Validation Accuracy on ax1
    color = 'tab:blue'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)', color=color)
    ax1.plot(df_metrics['Epoch'], df_metrics['Training Accuracy (%)'], label='Train Acc', color='tab:blue')
    ax1.plot(df_metrics['Epoch'], df_metrics['Validation Accuracy (%)'], label='Val Acc', color='tab:cyan')
    ax1.tick_params(axis='y', labelcolor=color)

    # Create a second y-axis for Training Loss
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Loss', color=color)
    ax2.plot(df_metrics['Epoch'], df_metrics['Training Loss'], label='Train Loss', color='tab:red')
    ax2.tick_params(axis='y', labelcolor=color)

    # Combine legends from both axes
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

    # Set plot title and layout
    plt.title(title)
    plt.tight_layout()

    # Save and display the plot
    plt.savefig(save_path)
    plt.show()

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim


class SimpleANN(nn.Module):

    def __init__(self,layer_sizes,activation=nn.ReLU,last_layer_activation=nn.ReLU,dropout=0):

        super(SimpleANN, self).__init__()
        self.layers = nn.ModuleList()

        for i in range(len(layer_sizes)-2):
          self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
          self.layers.append(nn.Dropout(dropout))
          self.layers.append(activation())

        self.layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        if last_layer_activation is not None:
         self.layers.append(nn.Dropout(dropout))
         self.layers.append(last_layer_activation())

    def forward(self, x):
        x = x.view(-1, np.prod(x.shape[1:])) # Flatten the input
        x = x.float()
        for layer in self.layers:
            x = layer(x)
        return x

In [5]:
# Set random seed for reproducibility
def set_seed(seed=2024):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(1998)

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f'Using device: {device}')

Using device: cpu


In [13]:
## Creating a tensor dataset ##
from torch.utils.data import DataLoader, TensorDataset
def TorchDataLoader(training_sequences, batch_size):
  context_words = [item[0] for item in training_sequences]  # List of [context]
  target_words = [item[1] for item in training_sequences]   # List of target words

  # Convert lists to tensors
  context_tensor = torch.tensor(context_words, dtype=torch.long)  # Shape: (num_samples, 3)
  target_tensor = torch.tensor(target_words, dtype=torch.long)    # Shape: (num_samples,)

  # Create a TensorDataset
  dataset = TensorDataset(context_tensor, target_tensor)

  # Create a DataLoader for batching
  batch_size = 4
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  # Iterate over the DataLoader
  for batch_context, batch_target in dataloader:
      print("Batch context:", batch_context)
      print("Batch target:", batch_target)
      # You can now use batch_context and batch_target for model training

  return dataloader

In [14]:
dummy_text = [
    ([2, 2, 2], 7), ([2, 2, 7], 8), ([2, 7, 8], 3), ([7, 8, 3], 9),
    ([8, 3, 9], 4), ([3, 9, 4], 5), ([9, 4, 5], 2), ([2, 2, 2], 2),
    ([2, 2, 2], 12), ([2, 2, 12], 4), ([2, 12, 4], 5), ([12, 4, 5], 13),
    ([4, 5, 13], 3), ([5, 13, 3], 14), ([13, 3, 14], 15), ([3, 14, 15], 2),
    ([2, 2, 2], 6), ([2, 2, 6], 16), ([2, 6, 16], 17), ([6, 16, 17], 18),
    ([16, 17, 18], 19), ([17, 18, 19], 20), ([18, 19, 20], 6), ([19, 20, 6], 21),
    ([20, 6, 21], 22), ([6, 21, 22], 2)
]

dataloader = TorchDataLoader(dummy_text, 4)

Batch context: tensor([[ 6, 16, 17],
        [ 2,  2,  2],
        [ 2,  6, 16],
        [12,  4,  5]])
Batch target: tensor([18,  2, 17, 13])
Batch context: tensor([[ 2,  2,  6],
        [ 9,  4,  5],
        [ 3, 14, 15],
        [ 2,  2, 12]])
Batch target: tensor([16,  2,  2,  4])
Batch context: tensor([[ 2,  2,  7],
        [ 6, 21, 22],
        [ 2,  2,  2],
        [20,  6, 21]])
Batch target: tensor([ 8,  2,  7, 22])
Batch context: tensor([[ 2,  2,  2],
        [16, 17, 18],
        [13,  3, 14],
        [ 8,  3,  9]])
Batch target: tensor([ 6, 19, 15,  4])
Batch context: tensor([[ 5, 13,  3],
        [ 4,  5, 13],
        [17, 18, 19],
        [ 7,  8,  3]])
Batch target: tensor([14,  3, 20,  9])
Batch context: tensor([[19, 20,  6],
        [ 2,  2,  2],
        [18, 19, 20],
        [ 2,  7,  8]])
Batch target: tensor([21, 12,  6,  3])
Batch context: tensor([[ 3,  9,  4],
        [ 2, 12,  4]])
Batch target: tensor([5, 5])


In [20]:
layer_sizes = [3,2,1]

model = SimpleANN(layer_sizes=layer_sizes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

number_of_epochs = 30

for epoch in range(number_of_epochs):
    for batch_context, batch_target in dataloader:
        print(dataloader)
        #FORWARD PASS:
        X = batch_context
        Y = batch_target
        X, Y = X.to(device), Y.to(device)
        outputs = model(X)  # Model output for X
        loss = criterion(outputs, Y) # Compute the loss between model output and Y

        #BACKWARD PASS (updating the model parameters):
        optimizer.zero_grad()  # Clear gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update model parameters

    print(f"Epoch [{epoch+1}/{number_of_epochs}], Loss: {loss.item():.4f}")

IndexError: Target 2 is out of bounds.